In [1]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'askcompany.settings'
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"  
# ㄴ django 3.0 이상에서 jupyter에서 장고 환경을 구성하기 위해서 필요한 설정임(https://docs.djangoproject.com/en/3.1/topics/async/). 

import django
django.setup()

In [2]:
from instagram.models import Post

In [4]:
# Cell

qs = Post.objects.all()
print(type(qs))
print(qs.query)
qs

<class 'django.db.models.query.QuerySet'>
SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post"


<QuerySet [<Post: 첫번째 메시지>, <Post: 두번째 메시지>, <Post: 세번째 메시지>]>

In [10]:
# qs = Post.objects.all().order_by('-id')  # id에 대한 오름차순 정렬
qs = Post.objects.all().order_by('-id')  # id에 대한 내림차순 정렬, 정렬 필드는 한두개만 넣어주는게 성능상 유리함.
print(qs.query)
qs

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" ORDER BY "instagram_post"."id" DESC


<QuerySet [<Post: 세번째 메시지>, <Post: 두번째 메시지>, <Post: 첫번째 메시지>]>

In [11]:
qs = Post.objects.all().order_by('-id')[:2]  # 데이터 두개만 가져옴
print(qs.query)
qs  # 실제 데이터 필요한 시점에 쿼리한다.(lazy하게 동작)

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" ORDER BY "instagram_post"."id" DESC LIMIT 2


<QuerySet [<Post: 세번째 메시지>, <Post: 두번째 메시지>]>

In [21]:
for post in qs:
#     print(post.id, post.message, post.created_at)
    print('id: {id}, message: {message} {created_at}'.format(**post.__dict__))
#     print(post.__dict__.keys())

id: 3, message: 세번째 메시지 2020-10-10 03:16:43.041442+00:00
id: 2, message: 두번째 메시지 2020-10-10 03:16:38.342724+00:00


In [29]:
query = '메시지' # 검색어
# qs = Post.objects.all().filter(message__icontains=query).order_by('-id')  # 최신 글 먼저

qs = Post.objects.all()
qs = qs.filter(message__icontains=query)
qs = qs.order_by('-id')  # Chaning 줄이 길어지면..

print(qs.query)
qs

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" WHERE "instagram_post"."message" LIKE %메시지% ESCAPE '\' ORDER BY "instagram_post"."id" DESC


<QuerySet [<Post: 세번째 메시지>, <Post: 두번째 메시지>, <Post: 첫번째 메시지>]>

In [34]:
qs = Post.objects.all()
qs.filter(id__lte=2)  # less than, less than equal, greater than, grather than equal

<QuerySet [<Post: 첫번째 메시지>, <Post: 두번째 메시지>]>

In [35]:
qs.first()

<Post: 첫번째 메시지>

In [37]:
qs.last()

<Post: 세번째 메시지>

In [40]:
qs.none().first()

In [41]:
from django.db.models import Q

In [43]:
qs = Post.objects.all()
qs = qs.filter(id__gte=2, message__icontains=query)  # AND조건
print(qs.query)

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" WHERE ("instagram_post"."id" >= 2 AND "instagram_post"."message" LIKE %메시지% ESCAPE '\')


In [47]:
qs = Post.objects.all()
# qs = qs.filter(Q(id__gte=2) & Q(message__icontains=query))  # Q를 이용한 AND조건
qs = qs.filter(Q(id__gte=2) | Q(message__icontains=query))  # Q를 이용한 OR조건
print(qs.query)

cond = Q(id__gte=2) | Q(message__icontains=query)
qs = qs.filter(cond)  # 별도로 cond분리해서 추가적인 조건 추가 가능
print(qs.query)

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" WHERE ("instagram_post"."id" >= 2 OR "instagram_post"."message" LIKE %메시지% ESCAPE '\')
SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" WHERE (("instagram_post"."id" >= 2 OR "instagram_post"."message" LIKE %메시지% ESCAPE '\') AND ("instagram_post"."id" >= 2 OR "instagram_post"."message" LIKE %메시지% ESCAPE '\'))
